# Genomic Analysis Tool

## Libraries and Modules:

In [4]:
#Libraries
import os
import json
from pprint import pprint

import csv
import re
import gzip
import pandas

import time
from datetime import date

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from scipy import stats
from operator import itemgetter
from lifelines.statistics import logrank_test

from guizero import App, PushButton, Slider
from guizero import Text, Window, TextBox, CheckBox, Box

## Functions

In [5]:
#Functions
def Set_Directory(): #Declare directory for program location
    os.chdir(ChDirbox.get())
def Load_Metadata():
    with open(Meta_box.get()) as data_file:    
        data = json.load(data_file)
    for n in range(len(data)):
        if not bool('TARGET' in data[n]['associated_entities'][0]['entity_submitter_id']):
            files.append([data[n]['associated_entities'][0]['case_id'],data[n]['file_id'],data[n]['file_name']])
def Load_ClinicalData():
    with open(Clinical_box.get()) as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t')
        firstline=True
        for row in tsvin:
            if firstline:
                firstline=False
            else:
                if not bool('TARGET' in str(row[1])):
                    clinical_data.append([row[0],row[4],row[12],row[13],row[15],row[24]])
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
def Corr_Analysis():
    #Read GUI inputs
    count_alive = 0
    count_dead = 0
    for i in range(len(FPKM_data)):
        for n in range(len(clinical_data)):
            for m in range(len(FPKM_data[i][1])):
                if clinical_data[n][0] == FPKM_data[i][1][m][0]:
                    if 'alive' in clinical_data[n][3] and not clinical_data[n][5]=='--':
                        count_alive = count_alive + 1
                        break
                    if 'dead' in clinical_data[n][3] and not clinical_data[n][4]=='--':
                        count_dead = count_dead + 1
                        break
    plot_data_alive = []
    plot_data_dead = []
    for i in range(len((labels))):
        plot_data_alive.append(labels[i][1])
        plot_data_dead.append(labels[i][1])
    for i in range(len(plot_data_alive)):
        plot_data_alive[i] = [plot_data_alive[i],[['0',0.0]]* int(count_alive/len(FPKM_data))] 
    for i in range(len(plot_data_dead)):
        plot_data_dead[i] = [plot_data_dead[i],[['0',0.0]]* int(count_dead/len(FPKM_data))] 
    c_alive = 0
    c_dead = 0
    count_alive = count_alive/len(FPKM_data)
    count_dead = count_dead/len(FPKM_data)
    for i in range(len(FPKM_data)):
        for n in range(len(clinical_data)):
            for m in range(len(FPKM_data[i][1])):
                if clinical_data[n][0] == FPKM_data[i][1][m][0]:
                    if 'alive' in clinical_data[n][3] and not clinical_data[n][5]=='--':
                        plot_data_alive[i][1][c_alive] = [float(clinical_data[n][5]),FPKM_data[i][1][m][1]]
                        c_alive = c_alive + 1
                        break
                    if 'dead' in clinical_data[n][3] and not clinical_data[n][4]=='--':
                        plot_data_dead[i][1][c_dead] = [float(clinical_data[n][4]),FPKM_data[i][1][m][1]]
                        c_dead = c_dead + 1
                        break
        c_alive = 0
        c_dead = 0
    if SinglePlotCheckBox.get_value() == 1:
        # Plot All Single plots and export names
        # First get labels and plot names
        label1txt = labels_box1.get()
        label2txt = labels_box2.get()
        corr_csv_name = corr_vals_box.get()
        label1 = []
        label2 = []
        nlabels = int(SinPltNum.get())
        plotlabelstxt = plotnames_box.get()
        plotlabels = []
        spearmans = []
        pearsons = []
        kendall = []
        p_spearmans = []
        p_pearsons = []
        p_kendall = [] 

        label1 = label1txt.split("\n",-1)
        label2 = label2txt.split("\n",-1)
        plotlabels = plotlabelstxt.split("\n",-1)
        
        label1.remove("")
        label2.remove("")
        plotlabels.remove("")

        spearmans = np.zeros((len(label1)),dtype=np.float)
        pearsons = np.zeros((len(label1)),dtype=np.float)
        kendall = np.zeros((len(label1)),dtype=np.float)
        p_spearmans = np.zeros((len(label1)),dtype=np.float)
        p_pearsons = np.zeros((len(label1)),dtype=np.float)
        p_kendall = np.zeros((len(label1)),dtype=np.float)
        
        for q in range(len(label1)):
            for w in range(len(labels)):
                for j in range(len(labels)):
                    if label1[q] == labels[w][0] and label2[q] == labels[j][0]:
                        x = []
                        y = []
                        

                        plt.cla()
                        for n in range(len(plot_data_alive[w][1])):
                            x.append(plot_data_alive[w][1][n][1])
                        for n in range(len(plot_data_dead[w][1])):
                            x.append(plot_data_dead[w][1][n][1])
                        for n in range(len(plot_data_alive[j][1])):
                            y.append(plot_data_alive[j][1][n][1])
                        for n in range(len(plot_data_dead[j][1])):
                            y.append(plot_data_dead[j][1][n][1]) 
                        fig = plt.plot(x,y,marker = '*', linestyle = 'none',markersize = 2)
                        #xall = xalive_T + xdead_T
                        #yall = yalive_T + ydead_T
                        pearsondf = pandas.DataFrame(data={'x':x,
                                                           'y':y})
                        z = np.abs(stats.zscore(pearsondf))
                        threshold = 3
                        Q1 = pearsondf.quantile(.25)
                        Q3 = pearsondf.quantile(.75)
                        IQR = Q3 - Q1
                        pearsondf_o = pearsondf[(z < 3).all(axis=1)]
                        pearsondf_out = pearsondf[~((pearsondf < (Q1 - 1.5 * IQR)) | (pearsondf > (Q3 + 1.5 * IQR))).any(axis=1)]
                        xpearsons = pearsondf_out['x']
                        ypearsons = pearsondf_out['y']
                        
                        spearmans[q], p_spearmans[q]= stats.spearmanr(x, y)
                        pearsons[q] , p_pearsons[q] = stats.pearsonr(xpearsons, ypearsons)
                        kendall[q]  , p_kendall[q]  = stats.kendalltau(x, y)
                        txtstr = 'n = %4s'%(count_alive+count_dead)
                        plt.text(int(max(x)*.8),int(max(y)*.8),txtstr)
                        plt.xlabel(label1[q]+ " (FPKM)")
                        plt.ylabel(label2[q]+ " (FPKM)")
                        plt.title("Gene/Protein Correlations")
                        plt.savefig(plotlabels[q],dpi = 300)
                        plt.close()
                        print("done")
                     
        #Create a Pandas dataframe of
        # 'CorrelationName' | 'Spearmans rho (Alive)' | 'p-value' | 'Spearmans rho (Dead)' | 'p-value' | 'Spearmans rho (All)' | p-value' 
        print(plotlabels[0])
        df = pandas.DataFrame(data={'Filename':label2,
                                    'Pearsons':pearsons,
                                    'p-value (P)':p_pearsons,
                                    'Spearmans rho':spearmans,
                                    'p-value (S)':p_spearmans,
                                    'Kendalls tau':kendall,
                                    'p-value (K)':p_kendall})
        df.to_csv(corr_csv_name,sep = ',',index = True)                                                                            
    if Large_plt_box.get_value() == 1:
        # Create matrix of Correlations
        # First get plot name
        Lplotlabel = LargePltbox.get()
        fig, axs = plt.subplots(len(labels), len(labels), figsize=(25,25))
        fig.subplots_adjust(hspace = .2, wspace = .2)
        axs = axs.ravel()
        pltcount = 0
        for i in range(len(labels)):
            for j in range(len(labels)):
                x=[]
                y=[]
                
                for n in range(len(plot_data_alive[i][1])):
                    x.append(plot_data_alive[i][1][n][1])
                for n in range(len(plot_data_dead[i][1])):
                    x.append(plot_data_dead[i][1][n][1])
                for n in range(len(plot_data_alive[j][1])):
                    y.append(plot_data_alive[j][1][n][1])
                for n in range(len(plot_data_dead[j][1])):
                    y.append(plot_data_dead[j][1][n][1])    
                axs[pltcount].plot(x,y,marker='*',linestyle='none')
                for n in range(len(labels)):
                    if plot_data_alive[j][0] in labels[n][1]:
                        axs[pltcount].set_ylabel(labels[n][0])
                        break
                for n in range(len(labels)):
                    if plot_data_alive[i][0] in labels[n][1]:
                        axs[pltcount].set_xlabel(labels[n][0])
                        break
                pltcount = pltcount + 1
        fig.savefig(Lplotlabel)
        plt.close(fig)
             
    if Heat_button.get_value() == 1:
        condexpOptions = Heatexport.get_value()
        Heatlabeltxt = Heatlabelbox.get()
        alive_pearsons = []
        dead_pearsons = []
        all_pearsons = []
        alive_p_values = []
        dead_p_values = []
        all_p_values = []
        diff_pearsons = []
        for i in range(len(labels)):
            alive_pearsons = np.zeros((len(label1),len(label1)),dtype=np.float)
            dead_pearsons = np.zeros((len(labels),len(labels)),dtype=np.float)
            all_pearsons = np.zeros((len(labels),len(labels)),dtype=np.float)
            diff_pearsons = np.zeros((len(labels), len(labels)), dtype = np.float)
            alive_p_values = np.zeros((len(labels),len(labels)),dtype=np.float)
            dead_p_values = np.zeros((len(labels),len(labels)),dtype=np.float)
            all_p_values = np.zeros((len(labels),len(labels)),dtype=np.float)
        for i in range(len(labels)):
            for j in range(len(labels)):
                xalive= []
                yalive= []
                xdead= []
                ydead= []
                xall= []
                yall= []
                for n in range(len(plot_data_alive[i][1])):
                    xalive.append(plot_data_alive[i][1][n][1])
                for n in range(len(plot_data_dead[i][1])):
                    xdead.append(plot_data_dead[i][1][n][1])
                for n in range(len(plot_data_alive[j][1])):
                    yalive.append(plot_data_alive[j][1][n][1])
                for n in range(len(plot_data_dead[j][1])):
                    ydead.append(plot_data_dead[j][1][n][1]) 
                xall = xalive + xdead
                yall = yalive + ydead
                alive_pearsons[i,j], alive_p_values[i,j]= stats.spearmanr(xalive, yalive)
                dead_pearsons[i,j] , dead_p_values[i,j] = stats.spearmanr(xdead, ydead)
                all_pearsons[i,j]  , all_p_values[i,j]  = stats.spearmanr(xall,yall)
                diff_pearsons[i,j] = abs(alive_pearsons[i,j] - dead_pearsons[i,j])
                
        plt.figure(figsize = (25,15))
        plt.subplot(1,2,1)

        # Generate a custom diverging colormap
        cmap = sns.diverging_palette(150, 10, s=80, l=55, as_cmap=True, center = 'dark')
        cmap_p = sns.diverging_palette(10,150,s=80,l=55, as_cmap=True)

        # Generate a mask for the upper triangle
        mask0 = np.zeros_like(alive_pearsons, dtype=np.bool)
        mask0[np.triu_indices_from(mask0)] = True
        sns.heatmap(all_pearsons,mask=mask0,cmap = cmap,square = True,vmin = -1, vmax = 1)
        plt.title('Spearmans Coefficients Rho')

        plt.subplot(2,3,2)
        mask1 = np.zeros_like(all_p_values, dtype=np.bool)
        mask1[np.triu_indices_from(mask1)] = True
        sns.heatmap(all_p_values,mask=mask1,cmap = cmap_p,square = True,vmin = 0, vmax = 1)
        plt.title('P-values')

#         plt.subplot(2,3,3)
#         mask2 = np.zeros_like(all_pearsons, dtype=np.bool)
#         mask2[np.triu_indices_from(mask2)] = True
#         sns.heatmap(all_pearsons,mask=mask2,cmap = cmap,square = True,vmin = -1, vmax = 1)
#         plt.title('Pearsons Coefficient for All Patients')

#         plt.subplot(2,3,4)
#         mask3 = np.zeros_like(alive_p_values, dtype=np.bool)
#         mask3[np.triu_indices_from(mask3)] = True
#         sns.heatmap(alive_p_values,mask=mask3,square = True,vmin = 0)
#         plt.title('P-Values for Alive Patients')

#         plt.subplot(2,3,5)
#         mask4 = np.zeros_like(dead_p_values, dtype=np.bool)
#         mask4[np.triu_indices_from(mask4)] = True
#         sns.heatmap(dead_p_values,mask=mask4,square = True,vmin = 0)
#         plt.title('P-Values for Dead Patients')

#         plt.subplot(2,3,6)
#         mask5 = np.zeros_like(all_p_values, dtype=np.bool)
#         mask5[np.triu_indices_from(mask5)] = True
#         sns.heatmap(all_p_values,mask=mask5,square = True,vmin = 0)
#         plt.title('P-Values for All Patients')

        plt.savefig(Heatlabeltxt)
        plt.close()
                

def Survival_Probability_Analysis():
    surv_labels = []
    
def close_app():
    app.destroy()
def open_Corr():
    Correlates.show(wait = True)
def close_Corr():
    Correlates.hide()   
def open_Surv():
    SurvProbOptions.show(wait = True)
def close_Surv():
    SurvProbOptions.hide()    
def open_IDs():
    IDs_input.show(wait = True)
def close_IDs():
    IDs_input.hide()    
def open_IDs_single():
    IDs_input_single.show(wait = True)
def close_IDs_single():
    IDs_input_single.hide()
def open_HeatOptions():
    HeatmapOptions.show(wait = True)
def close_HeatOptions():
    HeatmapOptions.hide()
def Load_FPKM():
    #Read GUI inputs
    nIDsa = int(nGeneProt_box.get())
    labellines = []
    labeltext = IDs_box.get()
    for i in range(nIDsa-1):
        labellines.append(labeltext.split("\n"))
    for i in range(nIDsa):
        labels.append(labellines[0][i].split(" "))
    
    #Read FPKM data
    for i in range(len((labels))):
        FPKM_data.append(labels[i][1])
    for i in range(len(FPKM_data)):
        FPKM_data[i] = [FPKM_data[i],[['0',0.0]]*len(files)]
    for n in range(len(files)):
        with gzip.open(FPKM_box.get()+files[n][1]+'/'+files[n][2], 'rb') as f:
            for line in f:
                for i in range(len(FPKM_data)):
                    if bool(FPKM_data[i][0] in str(line)):
                        string=str(line)
                        FPKM_data[i][1][n] = [files[n][0],float(string[string.find('\\t')+2:string.find('\\n')])]
def Read_data():
    Set_Directory()
    files = Load_Metadata()
    clinical_data = Load_ClinicalData()
    FPKM_data = Load_FPKM()
    print("Done Reading Data")
def Run():
    #Now read all parameters when run
    if checkbox1.get_value() == 1:
        Corr_Analysis()
    if checkbox2.get_value() == 1:
        Survival_Probability_Analysis()
    

## GUI -- RUN this cell to pull up command window

In [7]:
#Main Window
app = App(title = "Genomic Analysis using TCGA data")
#Sub Windows for list entrys
files = []
clinical_data = []
FPKM_data = []
labels = []
count_alive = 0
count_dead = 0

Correlates = Window(app, title = "Correlation Analysis Options",layout = "grid")
Correlates.hide()
IDs_input = Window(app, title = "Correlation ID Input")
IDs_input.hide()
IDs_input_single = Window(app, title = "Single Plot Correlations",layout = "grid")
IDs_input_single.hide()
HeatmapOptions = Window(app, title = "Heatmap Options",layout = "grid")
HeatmapOptions.hide()
SurvProbOptions = Window(app, title = "Survival Probability Plot Options", layout = "grid")
SurvProbOptions.hide()

#Main Window Layout - app
checkbox1 = CheckBox(app, text = "Correlation Analysis?") # Create command for when checked
Corr_button = PushButton(app, text = "Options",command =open_Corr)
checkbox2 = CheckBox(app, text = "Survival Probability Analysis")
Surv_button = PushButton(app, text = "Options",command = open_Surv) # Create command for when checked
ChDirtxt = Text(app, text = "Enter file path for current program location:")
ChDirbox = TextBox(app, text = "/Users/SamuelMulder/Desktop/TTU_Capstone/Kidney_Exp_Quant",width = 35)
Meta_txt = Text(app, text = "Enter name of Metadata file:")
Meta_box = TextBox(app, text = "Kidney_Meta_Data/metadata.cart.2018-11-06.json",width = 35)
Clinical_txt = Text(app, text = "Enter name of Clinical data file:")
Clinical_box = TextBox(app, text = "Kidney_Clinical_Data/clinical.cart.2018-10-31/clinical.tsv", width = 35)
FPKM_txt = Text(app, text = "Enter name of FPKM data folder:")
FPKM_box = TextBox(app, text = "Kidney_FPKM_Data/", width = 35)
nGeneProt = Text(app, text = "How many genes/proteins to analyze?", size = 18)
nGeneProt_box = TextBox(app, text = "8")#Create command for when text is changed
ID_button = PushButton(app, text = "Enter IDs",command = open_IDs)

buttons_box = Box(app, width="fill", align="bottom")
cancel = PushButton(buttons_box, text="Cancel", align="right",command = close_app)#Add values
Run = PushButton(buttons_box, text="Run", align="right",command = Run)#Add values
read_data = PushButton(buttons_box, text = "Read Data", align = "right", command = Read_data) #Add values

#Correlation Analysis Options - Correlates
SinglePlotCheckBox = CheckBox(Correlates, text = "Any single correlation plots?",grid=[1,2])
ID_button_single = PushButton(Correlates, text = "Export Options",command = open_IDs_single, grid = [1,3])
Large_plt_box = CheckBox(Correlates, text = "Large Combined Correlation Figure?", grid = [1,4])
LargePlottxt = Text(Correlates, text = "Plot file name?",grid=[1,5], size = 18)
LargePltbox = TextBox(Correlates, text = "/Users/SamuelMulder/Desktop/test.png",grid = [2,5])
Heat_button = CheckBox(Correlates, text = "Heatmap Analysis?",grid = [1,6])
Heatlabelbox = TextBox(Correlates, text = "Plot file name?",grid = [1,7])

buttons_box_corr = Box(Correlates, width="fill", grid = [2,10])
cancel = PushButton(buttons_box_corr, text="Cancel", align="right",command = close_Corr)
apply = PushButton(buttons_box_corr, text="Apply", align="right",command = close_Corr)

#IDs Input Window - IDs_input
IDs_text = Text(IDs_input, text = "Input IDs below:", align ="top", size = 18)
IDs_box = TextBox(IDs_input, text = "'label' 'ENSG#'",width = 30,height = 20,multiline = True,scrollbar = True)

buttons_box_IDs = Box(IDs_input, width="fill", align = "bottom")
cancel = PushButton(buttons_box_IDs, text="Cancel", align="right", command = close_IDs)
apply = PushButton(buttons_box_IDs, text="Apply", align="right",command = close_IDs)

#Single Plot Export Options - IDs_input_single
SinPltNumtxt = Text(IDs_input_single, text = "How many?",grid = [0,0], size = 18)
SinPltNum = TextBox(IDs_input_single, text = "Enter here",grid = [1,0])
labels_text = Text(IDs_input_single, text = "Input labels 1 and 2 below:", grid = [0,2], size = 18)
labels1_text = Text(IDs_input_single, text = "Label 1", grid = [0,3], size = 18)
labels2_text = Text(IDs_input_single, text = "Label 2",  grid = [1,3], size = 18)
labels_box1 = TextBox(IDs_input_single, text = "Enter 1 here",width = 20,multiline = True,scrollbar = True, grid = [0,4])
labels_box2 = TextBox(IDs_input_single, text = "Enter 2 here",width = 20,multiline = True,scrollbar = True, grid = [1,4])
plotnames_text = Text(IDs_input_single, text = "Enter plot names:",grid = [0,5], size = 18)
plotnames_box = TextBox(IDs_input_single, text = "/Users/SamuelMulder/Desktop/test.png",width = 40,multiline = True,scrollbar = True, grid = [0,6])
corr_vals_test = Text(IDs_input_single, text = "Enter csv filename for correlation values:",grid = [0,7], size = 18)                                               
corr_vals_box = TextBox(IDs_input_single, text = "/Users/SamuelMulder/Desktop/corr_vals.csv",width = 40,multiline = True,scrollbar = True, grid = [0,8])
                                                                                    
buttons_box_IDs = Box(IDs_input_single, width="fill", align = "bottom",grid = [1,10])
cancel = PushButton(buttons_box_IDs, text="Cancel", align="right", command = close_IDs_single)
apply = PushButton(buttons_box_IDs, text="Apply", align="right",command = close_IDs_single)

#Heatmap Export Options - HeatmapOptions
#Alivebox = CheckBox(HeatmapOptions, text = "Alive patients?",grid = [0,0])
#Deadbox = CheckBox(HeatmapOptions, text = "Dead patients?", grid = [0,1])
#Allbox = CheckBox(HeatmapOptions, text = "All patients?", grid = [0,2])
#Diffbox = CheckBox(HeatmapOptions, text = "Difference between Alive and Dead?", grid = [0,3])
#Heatexport = CheckBox(HeatmapOptions, text = "Combined figures or single figures?",grid = [0,4])
#Combinedtxt = Text(HeatmapOptions, text = "Combined plot name:",grid = [0,5], size = 18)
#Combinedbox = TextBox(HeatmapOptions, "example.png",grid = [1,5],width = 15)
#Aliveplttxt = Text(HeatmapOptions, text = "Alive plot name:",grid = [0,6], size = 18)
#Alivepltbox = TextBox(HeatmapOptions, text = "exmple.png",grid=[1,6],width = 15)
#Deadplttxt = Text(HeatmapOptions, text = "Dead plot name:",grid = [0,7], size = 18)
#Deadpltbox = TextBox(HeatmapOptions, text = "exmple.png",grid=[1,7],width = 15)
#Allplttxt = Text(HeatmapOptions, text = "All plot name:",grid = [0,8], size = 18)
#Allpltbox = TextBox(HeatmapOptions, text = "exmple.png",grid=[1,8],width = 15)

buttons_box_Heat = Box(HeatmapOptions, width="fill", align = "bottom",grid =[1,10])
cancel = PushButton(buttons_box_Heat, text="Cancel", align="right", command = close_HeatOptions)
apply = PushButton(buttons_box_Heat, text="Apply", align="right",command = close_HeatOptions)

#Survival Probability Analysis Options - SurvProbOptions
text1 = Text(SurvProbOptions, text = "How many Genes/Proteins?",grid = [0,0], size = 18)
NumSurvProb = TextBox(SurvProbOptions, text = "Enter Here", grid = [1,0])
text2 = Text(SurvProbOptions, text = "Enter labels:", grid = [0,1], size = 12)
text3 = Text(SurvProbOptions, text = "Enter filenames:", grid = [1,1], size = 12)
Survlabels = TextBox(SurvProbOptions, text = "Enter labels Here", width = 15, grid = [0,2],multiline = True, scrollbar = True, height = 10)
SurvPltnames = TextBox(SurvProbOptions, text = "Enter filenames Here", width = 20, grid = [1,2],multiline = True, scrollbar = True, height = 10)

buttons_box_Surv = Box(SurvProbOptions, width="fill", align = "bottom",grid =[2,10])
cancel = PushButton(buttons_box_Surv, text="Cancel", align="right", command = close_Surv)
apply = PushButton(buttons_box_Surv, text="Apply", align="right",command = close_Surv)


app.display()